(orc-chp-label)=

# ORC Combined Heat and Power

## Introduction

Subject of the following example of a more complex thermodynamic process will be a combined geothermal heat and power plant. As even deep geothermal boreholes can not supply temperatures high enough to drive conventional power plant processes, such as the aforementioned and thoroughly analyzed Clausius Rankine Cycle, adjustments are necessary. Organic process fluids whose vaporization temperature is lower than that of water allow the execution of a Rankine cycle at these lower heat source temperatures, resulting in the so-called *Organic Rankine Cycle*.

The following exercises are supposed to emulate a typical energy engineers workflow. The task is a process design study, where conditions of the surroundings and requirements of stakeholders are given. The proposed process design has to fulfill these constraints, which are displayed in {numref}`geothermal-chp-parameters`.

```{figure} /figures/geothermal_chp_parameters.svg
---
name: geothermal-chp-parameters
---
Boundary conditions of the geothermal plant with cogeneration of heat and power.
```

Additionally, {numref}`comp-approx` lists approximations of typical component parameters.

```{list-table} Approximations of typical component parameters
:header-rows: 1
:name: comp-approx

* - Component
  - Parameter
  - 
  - Value
  - Unit
* - Auxillary Installations
  - Electrical power requirements
  - 
  - 3
  - MW
* - Generators
  - Mechanical-electrical efficiency
  - 
  - 0.985
  - 
* - Motors
  - Electrical-mechanical efficiency
  - 
  - 0.975
  - 
* - Steam turbines
  - Isentropic efficiency
  - 
  - 0.9
  - 
* - 
  - Mechanical efficiency
  - 
  - 0.99
  - 
* - 
  - Minimum steam quality
  - 
  - 0.9
  - 
* - Pumps
  - Isentropic efficiency
  - 
  - 0.8
  - 
* - 
  - Mechanical efficiency
  - 
  - 0.99
  - 
* - Heat Exchangers
  - Minimal temperature difference
  - liquid-liquid
  - 10
  - K
* - 
  - 
  - liquid-condensing
  - 5
  - K
* - 
  - 
  - liquid-steam
  - 20
  - K
* - 
  - 
  - liquid-gaseous
  - 25
  - K
```

The proposed process is to be evaluated both with an energy- and an exergy-based approach. This can be achieved via efficiency values, which put the benefit of the process in relation to the necessary input. The energy efficiency of the combined heat and power process is defined according to Eq. {eq}`orc-chp-energy-efficiency` and the exergy efficiency according to Eq. {eq}`orc-chp-exergy-efficiency`.

```{math}
    :label: orc-chp-energy-efficiency
    \eta_{CHP} = \frac{\dot{W}_{net} + \dot{Q}_{DH}}{\dot{H}_1 - \dot{H}_2}
```

```{math}
    :label: orc-chp-exergy-efficiency
    \epsilon_{CHP} = \frac{\dot{W}_{net} + (\dot{E}_{ff} - \dot{E}_{bf})}{\dot{E}_1 - \dot{E}_2}
```


## Exercise 1

1. Build a geothermal combined heat and power plant according to the boundary conditions in {numref}`geothermal-chp-parameters` and {numref}`comp-approx`.
2. Build another geothermal plant based on the one before that replaces the heat output with another cycle extracting electrical power.
3. Evaluate both processes according to Eqs. {eq}`orc-chp-energy-efficiency` and {eq}`orc-chp-exergy-efficiency` and compare each other.

```{Note}

This task is described very openly, so there is no single right solution. You have to be creative to find a working process that fulfills all requirements.

At first, you can try to come up with components that are necessary to supply the desired outputs. You can build simple cycles and contemplate how they can be improved.

Additionally, you can get inspiration through existing plants as described in the literature. A few examples are {cite:t}`DiPippo2015`, {cite:t}`Drbal1996`, {cite:t}`Kitto2005` and {cite:t}`Strauss2016` (German).

```

```{note}

Your model very likely will be quite complex and use a lot of components which need parametrization. Building the whole model at once likely results in too few or too many parameters, which can be hard to track down. Oftentimes, a better approach is to build parts of the model, e.g. inner cycles or outer strands, and make them work. Only then the parts are joined together to build the whole process.

Also, you will probably encounter issues when setting terminal temperature differences without stable starting values. As mentioned in other examples, running the model with best-guess starting values before finalizing parametrization strengthens stability and can even be necessary. Besides the other examples, you are referred to the TESPy tutorial on <a href="https://tespy.readthedocs.io/en/dev/tutorials/starting_values.html#tespy-tutorial-starting-values-label">How to Generate Stable Starting Values</a>.

```

````{admonition} Proposed process designs (only use if necessary!)
:class: dropdown

If you are having trouble creating/finding a suitable process design that fulfills all the requirements, example designs are provided here. Definitely try to come up with your own process designs at first, as there is a lot to learn trying, failing and hopefully finally succeeding in solving the tasks.

For the cogeneration of heat and power, an ORC using the liquid phase from the geothermal source will be employed to solely supply power. The gaseous mass fraction can be fed directly through a steam turbine and condensed to supply the district heating network with heat. The process diagram of the described CHP plant can be found in {numref}`orc-chp-flowsheet`. The heat outflow to close the Organic Rankine Cycle is realized through cooling with ambient air. Finally, the two streams from the geothermal source are combined again to be fed back into the geothermal sink.

```{figure} /figures/geothermal_orc_chp.svg
---
name: orc-chp-flowsheet
---
Flow sheet of the geothermal plant with cogeneration of heat and power.
```

The second approach without the supply of heat is implemented with an additional Organic Rankine Cycle replacing the district heating condenser. This second ORC is in principle of the same structure as the first one, as can be seen in {numref}`orc-full-elec-flowsheet`.

```{figure} /figures/geothermal_orc_full_elec.svg
---
name: orc-full-elec-flowsheet
---
Flow sheet of the geothermal plant with sole power generation.
```

````

(orc-proposed-solution-1-label)=

## Proposed solution 1

**First Step**: Build the ORC as basis for geothermal combined heat and power plant and geothermal power plant

In [ ]:
from tespy.networks import Network
from tespy.connections import Connection, Bus
from tespy.components import (
    CycleCloser, Source, Sink, Pump, Turbine, HeatExchanger,HeatExchangerSimple,
    Condenser, Drum, DropletSeparator, Merge
    )

# Network
nw = Network(fluids=['water', 'air', 'Isopentane'])
nw.set_attr(T_unit='C', p_unit='bar', h_unit='kJ / kg')

flu_vec_orc = {'water': 0, 'air': 0, 'Isopentane': 1}
flu_vec_water = {'water': 1, 'air': 0, 'Isopentane': 0}
flu_vec_air = {'water': 0, 'air': 1, 'Isopentane': 0}

# Components
# ORC
orc_cc = CycleCloser('ORC Cycle Closer')
orc_preheater = HeatExchanger('ORC Preheater')
orc_drum = Drum('ORC Drum')
orc_evaporator = HeatExchanger('ORC Evaporator')
orc_turbine = Turbine('ORC Turbine')
orc_condenser = Condenser('ORC Condenser')
orc_pump = Pump('ORC Pump')

# Condenser coolant cycle
cool_source = Source('Coolant Source')
cool_sink = Sink('Coolant Sink')

# Auxiliary source and sink
liquid_source = Source('Liquid Sep Source')
geo_pump = Pump('Geothermal Pump')
geo_sink1 = Sink('Geothermal Sink1')

# Connections
# ORC
orc_pump2orc_cc = Connection(orc_pump, 'out1', orc_cc, 'in1', label='30')
orc_cc2orc_preheater = Connection(orc_cc, 'out1', orc_preheater, 'in2', label='31')
orc_preheat2orc_drum = Connection(orc_preheater, 'out2', orc_drum, 'in1', label='32')
orc_drum2orc_eva = Connection(orc_drum, 'out1', orc_evaporator, 'in2', label='33')
orc_eva2orc_drum = Connection(orc_evaporator, 'out2', orc_drum, 'in2', label='34')
orc_drum2orc_turb = Connection(orc_drum, 'out2', orc_turbine, 'in1', label='35')
orc_turb2orc_cond = Connection(orc_turbine, 'out1', orc_condenser, 'in1', label='36')
orc_cond2orc_pump = Connection(orc_condenser, 'out1', orc_pump, 'in1', label='37')

nw.add_conns(
    orc_pump2orc_cc, orc_cc2orc_preheater, orc_preheat2orc_drum, orc_drum2orc_eva,
    orc_eva2orc_drum, orc_drum2orc_turb, orc_turb2orc_cond, orc_cond2orc_pump
    )

# Condenser coolant strand
cool_source2orc_cond = Connection(cool_source, 'out1', orc_condenser, 'in2', label='51')
orc_cond2cool_sink = Connection(orc_condenser, 'out2', cool_sink, 'in1', label='52')

nw.add_conns(cool_source2orc_cond, orc_cond2cool_sink)

# Geothermal liquid strand
liquid_source2orc_eva = Connection(liquid_source, 'out1', orc_evaporator, 'in1', label='17')
orc_eva2orc_preheat = Connection(orc_evaporator, 'out1', orc_preheater, 'in1', label='18')
orc_preheat2geo_pump = Connection(orc_preheater, 'out1', geo_pump, 'in1', label='19')
geo_pump2geo_sink1 = Connection(geo_pump, 'out1', geo_sink1, 'in1', label='21')

nw.add_conns(
    liquid_source2orc_eva, orc_eva2orc_preheat, orc_preheat2geo_pump, geo_pump2geo_sink1
    )

**Second Step**: Set the process parameters and starting values. After that solve orc cycle with starting and final values

In [ ]:
from CoolProp.CoolProp import PropsSI as PSI

# Parametrization
# Component parameters
orc_turbine.set_attr(eta_s=0.9)

orc_pump.set_attr(eta_s=0.8)
geo_pump.set_attr(eta_s=0.8)

orc_preheater.set_attr(pr1=1, pr2=1)
orc_evaporator.set_attr(pr1=1)
orc_condenser.set_attr(pr1=1, pr2=1)

# Connection parameter
# ORC
p_orc_condenser = PSI("P", "Q", 0, "T", 35 + 273.15, 'Isopentane') * 1e-5
orc_turb2orc_cond.set_attr(p=p_orc_condenser, fluid=flu_vec_orc)

p_orc_evaporator = PSI("P", "Q", 1, "T", 150 + 273.15, 'Isopentane') * 1e-5
orc_eva2orc_drum.set_attr(p=p_orc_evaporator, x=0.7)

orc_preheat2orc_drum.set_attr(Td_bp=-5)

# Geothermal liquid strand
liquid_source2orc_eva.set_attr(m=500*0.55, p=10.8, x=0, fluid=flu_vec_water)
geo_pump2geo_sink1.set_attr(T=55, p=30)

# Condenser coolant strand
cool_source2orc_cond.set_attr(T=25, p=1.013, fluid=flu_vec_air)
orc_cond2cool_sink.set_attr(T=30)

# Initial solve with starting values
nw.solve("design")
nw.print_results()

# Final solve
orc_eva2orc_drum.set_attr(p=None)
orc_evaporator.set_attr(ttd_l=10)

orc_turb2orc_cond.set_attr(p=None)
orc_condenser.set_attr(ttd_u=5)

nw.solve("design")
nw.print_results()

## Proposed solution 1.1

### Geothermal combined heat and power plant

**First Step**: Build the geothermal steam strand for the combined heat and power process design

Be sure this one runs standalone first!

In [ ]:
# Geothermal steam strand with district heating system
# Components
steam_turbine = Turbine('Steam Turbine')
dh_condenser = Condenser('District Heating Condenser')
dh_preheater = HeatExchanger('District Heating Preheater')
dh_cc = CycleCloser('District Heating Cycle Closer')
dh_pump = Pump('District Heating Pump')
dh_heatsink = HeatExchangerSimple('District Heating Consumer')
steam_pump = Pump('Steam Pump')

# Auxiliary source and sink
steam_source = Source('Steam Sep Source')
geo_sink2 = Sink('Geothermal Sink2')

# Connection
# Geothermal steam strand
steam_source2steam_turb = Connection(steam_source, 'out1', steam_turbine, 'in1', label='12')
steam_turb2dh_cond = Connection(steam_turbine, 'out1', dh_condenser, 'in1', label='13')
dh_cond2dh_preheat = Connection(dh_condenser, 'out1', dh_preheater, 'in1', label='14')
dh_preheat2steam_pump = Connection(dh_preheater, 'out1', steam_pump, 'in1', label='15')
steam_pump2geo_sink2 = Connection(steam_pump, 'out1', geo_sink2, 'in1', label='16')

nw.add_conns(
    steam_source2steam_turb, steam_turb2dh_cond, dh_cond2dh_preheat,
    dh_preheat2steam_pump, steam_pump2geo_sink2
    )

# District heating system
dh_heatsink2dh_cc = Connection(dh_heatsink, 'out1', dh_cc, 'in1', label='60')
dh_cc2dh_preheat = Connection(dh_cc, 'out1', dh_preheater, 'in2', label='61')
dh_preheat2dh_cond = Connection(dh_preheater, 'out2', dh_condenser, 'in2', label='62')
dh_cond2dh_heatsink = Connection(dh_condenser, 'out2', dh_heatsink, 'in1', label='63')

nw.add_conns(
    dh_heatsink2dh_cc, dh_cc2dh_preheat, dh_preheat2dh_cond,
    dh_cond2dh_heatsink
    )

# Parametrization
# Components parameter
steam_turbine.set_attr(eta_s=0.9)
steam_pump.set_attr(eta_s=0.8)

dh_condenser.set_attr(pr1=1, pr2=1)
dh_preheater.set_attr(pr1=1, pr2=1)

# Connection parameter
# Geothermal steam strand
p_dh_condenser = PSI("P", "Q", 0, "T", 105 + 273.15, 'water') * 1e-5
steam_turb2dh_cond.set_attr(p=p_dh_condenser)

steam_source2steam_turb.set_attr(m=500*0.45, p=10.8, x=1, fluid=flu_vec_water)
steam_pump2geo_sink2.set_attr(T=55, p=30)

# District heating system
dh_cond2dh_heatsink.set_attr(T=100, p=10, fluid=flu_vec_water)
dh_heatsink2dh_cc.set_attr(T=50)

nw.solve('design')

steam_turb2dh_cond.set_attr(p=None)
dh_condenser.set_attr(ttd_u=5)

nw.solve('design')

**Second Step**: Connecting ORC with Steam Cycle

1. Build up the merge part
2. Build up the separator part
3. Set busses and solve the geothermal combined heat and power plant
4. Create and execute `ExergyAnalysis` instance

In [ ]:
from tespy.tools import ExergyAnalysis

# Components
# Geothermal heat source with interfaces
geo_source = Source('Geothermal Source')
geo_sink = Sink('Geothermal Sink')
seperator = DropletSeparator('Seperator')
merge = Merge('Merge')

# Build up the merge part
# Delete old connections
nw.del_conns(geo_pump2geo_sink1, steam_pump2geo_sink2, orc_preheat2geo_pump)

# Set new connections
steam_pump2merge = Connection(steam_pump, 'out1', merge, 'in1', label='16')
orc_preheat2merge = Connection(orc_preheater, 'out1', merge, 'in2', label='19')
merge2geo_pump = Connection(merge, 'out1', geo_pump, 'in1', label='20')
geo_pump2geo_sink = Connection(geo_pump, 'out1', geo_sink, 'in1', label='21')

nw.add_conns(steam_pump2merge, orc_preheat2merge, merge2geo_pump, geo_pump2geo_sink)

# Parametrize new connections
orc_preheat2merge.set_attr(T=55)
geo_pump2geo_sink.set_attr(p=30)
dh_preheat2steam_pump.set_attr(T=55)

# Build up the seperator part
# Delete old connections
nw.del_conns(liquid_source2orc_eva, steam_source2steam_turb)

# Set new connections
geo_source2sep = Connection(geo_source,'out1', seperator, 'in1', label='11')
sep2steam_turb = Connection(seperator, 'out2', steam_turbine, 'in1', label='12')
sep2orc_eva = Connection(seperator,'out1', orc_evaporator, 'in1', label='17')

nw.add_conns(geo_source2sep, sep2orc_eva, sep2steam_turb)

# Parametrize new connections
geo_source2sep.set_attr(m=500, p=10.8, x=0.45, fluid=flu_vec_water)
sep2orc_eva.set_attr(m0=500*0.55)
sep2steam_turb.set_attr(m0=500*0.45)

# Busses
heat_in = Bus('Heat Input')
heat_in.add_comps(
    {'comp': geo_source, 'base': 'bus'},
    {'comp': geo_sink, 'base': 'component'}
    )

power_out = Bus('Power Output')
power_out.add_comps(
    {'comp': steam_turbine, 'char': 0.985*0.99, 'base': 'component'},
    {'comp': orc_turbine, 'char': 0.985*0.99, 'base': 'component'},
    {'comp': steam_pump, 'char': 0.975*0.99, 'base': 'bus'},
    {'comp': orc_pump, 'char': 0.975*0.99, 'base': 'bus'},
    {'comp': geo_pump, 'char': 0.975*0.99, 'base': 'bus'}
    )

heat_out_dh = Bus('Heat Output District Heating')
heat_out_dh.add_comps({'comp': dh_heatsink, 'base': 'component'})

heat_out_cool = Bus('Heat Output Cooling')
heat_out_cool.add_comps(
    {'comp': cool_source, 'base': 'bus'},
    {'comp': cool_sink, 'base': 'component'}
    )

nw.add_busses(heat_in, power_out, heat_out_dh, heat_out_cool)

# Solve model
nw.solve("design")
nw.print_results()

# Calculate thermal efficiency
h_in = PSI('H', 'Q', 0.45, 'P', 10.8e5, 'water')
h_out = PSI('H', 'T', 55 + 273.15, 'P', 30e5, 'water')
delta_H = 500 * (h_in - h_out)
eta_el = abs(power_out.P.val)/delta_H
eta_th = abs(heat_out_dh.P.val)/delta_H
eta_chp = (abs(power_out.P.val)+abs(heat_out_dh.P.val))/delta_H
print(f'eta_el = {eta_el:.3f}')
print(f'eta_th = {eta_th:.3f}')
print(f'eta_chp = {eta_chp:.3f}')

# # Create and execute exergy analysis
dh_heatsink.set_attr(dissipative=False)
ean = ExergyAnalysis(
    network=nw, E_F=[heat_in], E_P=[heat_out_dh, power_out], E_L=[heat_out_cool])
ean.analyse(pamb=1.013, Tamb=25)
ean.print_results(components=False, connections=False, busses=False, groups=False)
epsilon_chp = ean.network_data['epsilon']
print(f'epsilon_chp = {epsilon_chp:.3f}')

## Proposed solution 1.2

### Geothermal plant with only power generation

**First Step**: Build low pressure ORC for complete condensation operation

Be sure this one runs standalone first!

In [ ]:
nw_el = Network(fluids=['water', 'air', 'Isopentane'])
nw_el.set_attr(T_unit='C', p_unit='bar', h_unit='kJ / kg')

flu_vec_orc = {'water': 0, 'air': 0, 'Isopentane': 1}
flu_vec_water = {'water': 1, 'air': 0, 'Isopentane': 0}
flu_vec_air = {'water': 0, 'air': 1, 'Isopentane': 0}

# Components
# Low pressure ORC
lp_orc_cc = CycleCloser('LP ORC Cycle Closer')
lp_orc_preheater = HeatExchanger('LP ORC Preheater')
lp_orc_drum = Drum('LP ORC Drum')
lp_orc_evaporator = HeatExchanger('LP ORC Evaporator')
lp_orc_turbine = Turbine('LP ORC Turbine')
lp_orc_condenser = Condenser('LP ORC Condenser')
lp_orc_pump = Pump('LP ORC Pump')

# Condenser coolant cycle
lp_cool_source = Source('LP Coolant Source')
lp_cool_sink = Sink('LP Coolant Sink')

# Geothermal steam Cycle
steam_turbine = Turbine('Steam Turbine')
steam_pump = Pump('Steam Pump')

# Auxiliary source and sink
steam_source = Source('Steam Sep Source')
geo_sink2 = Sink('Geothermal Sink2')

# Connections
# Low pressure ORC
lp_orc_pump2lp_orc_cc = Connection(lp_orc_pump, 'out1', lp_orc_cc, 'in1', label='40')
lp_orc_cc2lp_orc_preheater = Connection(lp_orc_cc, 'out1', lp_orc_preheater, 'in2', label='41')
lp_orc_preheat2lp_orc_drum = Connection(lp_orc_preheater, 'out2', lp_orc_drum, 'in1', label='42')
lp_orc_drum2lp_orc_eva = Connection(lp_orc_drum, 'out1', lp_orc_evaporator, 'in2', label='43')
lp_orc_eva2lp_orc_drum = Connection(lp_orc_evaporator, 'out2', lp_orc_drum, 'in2', label='44')
lp_orc_drum2lp_orc_turb = Connection(lp_orc_drum, 'out2', lp_orc_turbine, 'in1', label='45')
lp_orc_turb2lp_orc_cond = Connection(lp_orc_turbine, 'out1', lp_orc_condenser, 'in1', label='46')
lp_orc_cond2lp_orc_pump = Connection(lp_orc_condenser, 'out1', lp_orc_pump, 'in1', label='47')

nw_el.add_conns(
    lp_orc_pump2lp_orc_cc, lp_orc_cc2lp_orc_preheater, lp_orc_preheat2lp_orc_drum, 
    lp_orc_drum2lp_orc_eva, lp_orc_eva2lp_orc_drum, lp_orc_drum2lp_orc_turb, 
    lp_orc_turb2lp_orc_cond, lp_orc_cond2lp_orc_pump
    )

# Condenser coolant cycle
lp_cool_source2lp_orc_cond = Connection(lp_cool_source, 'out1', lp_orc_condenser, 'in2', label='71')
lp_orc_cond2lp_cool_sink = Connection(lp_orc_condenser, 'out2', lp_cool_sink, 'in1', label='72')

nw_el.add_conns(lp_cool_source2lp_orc_cond, lp_orc_cond2lp_cool_sink)

# Steam cyclye
steam_source2steam_turb = Connection(steam_source, 'out1', steam_turbine, 'in1', label='12')
steam_turb2lp_orc_eva = Connection(steam_turbine, 'out1', lp_orc_evaporator, 'in1', label='13')
lp_orc_eva2lp_orc_preheat = Connection(lp_orc_evaporator, 'out1', lp_orc_preheater, 'in1', label='14')
lp_orc_preheat2steam_pump = Connection(lp_orc_preheater, 'out1', steam_pump, 'in1', label='15')
steam_pump2geo_sink2 = Connection(steam_pump, 'out1', geo_sink2, 'in1', label='16')

nw_el.add_conns(
    steam_source2steam_turb, steam_turb2lp_orc_eva, lp_orc_eva2lp_orc_preheat,
    lp_orc_preheat2steam_pump, steam_pump2geo_sink2
    )

**Second Step**: Set the process parameters and starting values. After that solve orc cycle with starting and final values

In [ ]:
from CoolProp.CoolProp import PropsSI as PSI

# Parametrization
# Component parameters
steam_turbine.set_attr(eta_s=0.9)
lp_orc_turbine.set_attr(eta_s=0.9)

lp_orc_pump.set_attr(eta_s=0.8)
steam_pump.set_attr(eta_s=0.8)

lp_orc_preheater.set_attr(pr1=1, pr2=1)
lp_orc_evaporator.set_attr(pr1=1)
lp_orc_condenser.set_attr(pr1=1, pr2=1)

# Connection parameter
# Low pressure ORC
p_lp_orc_condenser = PSI("P", "Q", 0, "T",  35 + 273.15, 'Isopentane') * 1e-5
lp_orc_turb2lp_orc_cond.set_attr(p=p_lp_orc_condenser, fluid=flu_vec_orc)

p_lp_orc_evaporator = PSI("P", "Q", 1, "T", 100 + 273.15, 'Isopentane') * 1e-5
lp_orc_eva2lp_orc_drum.set_attr(p=p_lp_orc_evaporator, x=0.5)

lp_orc_preheat2lp_orc_drum.set_attr(Td_bp=-5)

# Geothermal steam cyclye
steam_source2steam_turb.set_attr(m=500*0.45, p=10.8, x=0, fluid=flu_vec_water)
p_steam_lp_orc_evap = PSI("P", "Q", 0, "T",  120 + 273.15, 'water') * 1e-5
steam_turb2lp_orc_eva.set_attr(p=p_steam_lp_orc_evap)
# lp_orc_eva2lp_orc_preheat.set_attr(x0=0)
steam_pump2geo_sink2.set_attr(T=55, p=30)

# Condenser coolant cycle
lp_cool_source2lp_orc_cond.set_attr(T=25, p=1.013, fluid=flu_vec_air)
lp_orc_cond2lp_cool_sink.set_attr(T=30)

# Initial solve with starting values
nw_el.solve("design")

# Final solve
lp_orc_eva2lp_orc_drum.set_attr(p=None)
lp_orc_evaporator.set_attr(ttd_l=10)

lp_orc_turb2lp_orc_cond.set_attr(p=None)
lp_orc_condenser.set_attr(ttd_u=5)

nw_el.solve("design")

**Third Step**: *Identically* build the ORC as in {ref}`orc-proposed-solution-1-label`

In [ ]:
# Components
# ORC
orc_cc = CycleCloser('ORC Cycle Closer')
orc_preheater = HeatExchanger('ORC Preheater')
orc_drum = Drum('ORC Drum')
orc_evaporator = HeatExchanger('ORC Evaporator')
orc_turbine = Turbine('ORC Turbine')
orc_condenser = Condenser('ORC Condenser')
orc_pump = Pump('ORC Pump')

# Condenser coolant cycle
cool_source = Source('Coolant Source')
cool_sink = Sink('Coolant Sink')

# Auxiliary source and sink
liquid_source = Source('Liquid Sep Source')
geo_pump = Pump('Geothermal Pump')
geo_sink1 = Sink('Geothermal Sink1')

# Connections
# ORC
orc_pump2orc_cc = Connection(orc_pump, 'out1', orc_cc, 'in1', label='30')
orc_cc2orc_preheater = Connection(orc_cc, 'out1', orc_preheater, 'in2', label='31')
orc_preheat2orc_drum = Connection(orc_preheater, 'out2', orc_drum, 'in1', label='32')
orc_drum2orc_eva = Connection(orc_drum, 'out1', orc_evaporator, 'in2', label='33')
orc_eva2orc_drum = Connection(orc_evaporator, 'out2', orc_drum, 'in2', label='34')
orc_drum2orc_turb = Connection(orc_drum, 'out2', orc_turbine, 'in1', label='35')
orc_turb2orc_cond = Connection(orc_turbine, 'out1', orc_condenser, 'in1', label='36')
orc_cond2orc_pump = Connection(orc_condenser, 'out1', orc_pump, 'in1', label='37')

nw_el.add_conns(
    orc_pump2orc_cc, orc_cc2orc_preheater, orc_preheat2orc_drum, orc_drum2orc_eva,
    orc_eva2orc_drum, orc_drum2orc_turb, orc_turb2orc_cond, orc_cond2orc_pump
    )

# Condenser coolant cycle
cool_source2orc_cond = Connection(cool_source, 'out1', orc_condenser, 'in2', label='51')
orc_cond2cool_sink = Connection(orc_condenser, 'out2', cool_sink, 'in1', label='52')

nw_el.add_conns(cool_source2orc_cond, orc_cond2cool_sink)

# Geothermal liquid cyclye
liquid_source2orc_eva = Connection(liquid_source, 'out1', orc_evaporator, 'in1', label='17')
orc_eva2orc_preheat = Connection(orc_evaporator, 'out1', orc_preheater, 'in1', label='18')
orc_preheat2geo_pump = Connection(orc_preheater, 'out1', geo_pump, 'in1', label='19')
geo_pump2geo_sink1 = Connection(geo_pump, 'out1', geo_sink1, 'in1', label='21')

nw_el.add_conns(
    liquid_source2orc_eva, orc_eva2orc_preheat, orc_preheat2geo_pump, geo_pump2geo_sink1
    )

# Parametrization
# Component parameters
orc_turbine.set_attr(eta_s=0.9)

orc_pump.set_attr(eta_s=0.8)
geo_pump.set_attr(eta_s=0.8)

orc_preheater.set_attr(pr1=1, pr2=1)
orc_evaporator.set_attr(pr1=1)
orc_condenser.set_attr(pr1=1, pr2=1)

# Connection parameter
# ORC
p_orc_condenser = PSI("P", "Q", 0, "T", 35 + 273.15, 'Isopentane') * 1e-5
orc_turb2orc_cond.set_attr(p=p_orc_condenser, fluid=flu_vec_orc)

p_orc_evaporator = PSI("P", "Q", 1, "T", 150 + 273.15, 'Isopentane') * 1e-5
orc_eva2orc_drum.set_attr(p=p_orc_evaporator, x=0.7)

orc_preheat2orc_drum.set_attr(Td_bp=-5)

# Geothermal liquid cyclye
liquid_source2orc_eva.set_attr(m=500*0.55, p=10.8, x=0, fluid=flu_vec_water)
geo_pump2geo_sink1.set_attr(T=55, p=30)

# Condenser coolant cycle
cool_source2orc_cond.set_attr(T=25, p=1.013, fluid=flu_vec_air)
orc_cond2cool_sink.set_attr(T=30)

# Initial solve with starting values
nw_el.solve("design")

# Final solve
orc_eva2orc_drum.set_attr(p=None)
orc_evaporator.set_attr(ttd_l=10)

orc_turb2orc_cond.set_attr(p=None)
orc_condenser.set_attr(ttd_u=5)

nw_el.solve("design")

**Fourth Step**: Connecting ORC with low pressure ORC

1. Build the merge part
2. Build the separator part
3. Set busses and solve the geothermal combined heat and power plant
4. Create and execute `ExergyAnalysis` instance

In [ ]:
# Components
# Geothermal heat source with interfaces
geo_source = Source('Geothermal Source')
geo_sink = Sink('Geothermal Sink')
seperator = DropletSeparator('Seperator')
merge = Merge('Merge')

# Build up the merge part
# Delete old connections
nw_el.del_conns(geo_pump2geo_sink1, steam_pump2geo_sink2, orc_preheat2geo_pump)

# Set new connections
steam_pump2merge = Connection(steam_pump, 'out1', merge, 'in1', label='16')
orc_preheat2merge = Connection(orc_preheater, 'out1', merge, 'in2', label='19')
merge2geo_pump = Connection(merge, 'out1', geo_pump, 'in1', label='20')
geo_pump2geo_sink = Connection(geo_pump, 'out1', geo_sink, 'in1', label='21')

nw_el.add_conns(steam_pump2merge, orc_preheat2merge, merge2geo_pump, geo_pump2geo_sink)

# Parametrize new connections
orc_preheat2merge.set_attr(T=55)
geo_pump2geo_sink.set_attr(p=30)
lp_orc_preheat2steam_pump.set_attr(T=55)

# Build up the seperator part
# Delete old connections
nw_el.del_conns(liquid_source2orc_eva, steam_source2steam_turb)

# Set new connections
geo_source2sep = Connection(geo_source,'out1', seperator, 'in1', label='11')
sep2steam_turb = Connection(seperator, 'out2', steam_turbine, 'in1', label='12')
sep2orc_eva = Connection(seperator,'out1', orc_evaporator, 'in1', label='17')

nw_el.add_conns(geo_source2sep, sep2steam_turb, sep2orc_eva)

# Parametrize new connections
geo_source2sep.set_attr(m=500, p=10.8, x=0.45, fluid=flu_vec_water)
sep2orc_eva.set_attr(m0=500*0.55)
sep2steam_turb.set_attr(m0=500*0.45)

# Busses
heat_in = Bus('Heat Input')
heat_in.add_comps(
    {'comp': geo_source, 'base': 'bus'},
    {'comp': geo_sink, 'base': 'component'}
    )

power_out = Bus('Power Output')
power_out.add_comps(
    {'comp': steam_turbine, 'char': 0.985*0.99, 'base': 'component'},
    {'comp': orc_turbine, 'char': 0.985*0.99, 'base': 'component'},
    {'comp': lp_orc_turbine, 'char': 0.985*0.99, 'base': 'component'},
    {'comp': steam_pump, 'char': 0.975*0.99, 'base': 'bus'},
    {'comp': orc_pump, 'char': 0.975*0.99, 'base': 'bus'},
    {'comp': lp_orc_pump, 'char': 0.975*0.99, 'base': 'bus'},
    {'comp': geo_pump, 'char': 0.975*0.99, 'base': 'bus'}
    )

heat_out_cool = Bus('Heat Output Cooling')
heat_out_cool.add_comps(
    {'comp': cool_source, 'base': 'bus'},
    {'comp': cool_sink, 'base': 'component'},
    {'comp': lp_cool_source, 'base': 'bus'},
    {'comp': lp_cool_sink, 'base': 'component'}
    )

nw_el.add_busses(heat_in, power_out, heat_out_cool)

# Solve model
nw_el.solve("design")
nw_el.print_results()

# Calculate thermal efficiency
eta_el = abs(power_out.P.val)/delta_H
print(f'eta_el = {eta_el:.3f}')

# Create and execute exergy analysis
ean_el = ExergyAnalysis(
    network=nw_el, E_F=[heat_in], E_P=[power_out], E_L=[heat_out_cool])
ean_el.analyse(pamb=1.013, Tamb=25)
ean_el.print_results(components=False, connections=False, busses=False, groups=False)
epsilon_el = ean_el.network_data['epsilon']
print(f'epsilon_el = {epsilon_el:.3f}')

## Proposed solution 1.3

**First Step**: Evaluate both processes according to Eqs. {eq}`orc-chp-energy-efficiency` and {eq}`orc-chp-exergy-efficiency`

````{admonition} Comparison of energy and exergy efficiency
:class: dropdown

```{list-table} Comparison of energy and exergy efficiency of geothermal plants
:header-rows: 1
:name: result-comparison

* - 
  - Combined heat and power
  - Only power generation
* - $\eta_{CHP}$
  - 80.4 %
  - 19.0 %
* - $\epsilon_{CHP}$
  - 70.3 %
  - 62.8 %
```

````

**Second Step**: Compare both processes with a Grassmann diagram

In [ ]:
import plotly.graph_objects as go

links, nodes = ean.generate_plotly_sankey_input()

fig = go.Figure(
    go.Sankey(
        arrangement="snap",
        node={"label": nodes, 'pad': 11, 'color': 'orange'},
        link=links
        ),
    layout=go.Layout({'width': 1200})
    )
# plot(fig, filename='chp_grassmann')
fig.show()

links_el, nodes_el = ean_el.generate_plotly_sankey_input()

fig_el = go.Figure(
    go.Sankey(
        arrangement="snap",
        node={"label": nodes_el, 'pad': 11, 'color': 'orange'},
        link=links_el
        ),
    layout=go.Layout({'width': 1200})
    )

fig_el.show()

```{admonition} Explanation
:class: dropdown

Hier könnte ihre Erklärung stehen

- Weitere Übung. Analyse der Exergie bei den einzlnen Prozessen
```

## Lessons Learned

- Lessons Learned 1
- Lessons Learned 2
- Lessons Learned 3